# Extract night HRV

In [1]:
import numpy as np
import pandas as pd
import os
import glob

import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['lines.linewidth'] = 0.91
plt.style.use('seaborn-v0_8-whitegrid')
# %matplotlib qt

### Load processed data (from procees_empatica.ipynb)

In [2]:
processed_data_path = "/Users/augenpro/Documents/Empatica/data_sara/data/GGIR_input/"

ppg_df = pd.read_parquet(processed_data_path + "ppg.parquet")
temp_df = pd.read_parquet(processed_data_path + "temp.parquet")
acc_df = pd.read_parquet(processed_data_path + "acc.parquet")
nights = np.load(processed_data_path + "SPT_window_GGIR.npy", allow_pickle=True)

In [3]:
from utils.compute_acc_metrics import compute_acc_SMV
from sleep.detect_acc_bursts import *
from heart_rate.ppg_beat_detection import MSPTDfast
from heart_rate.kubios import signal_fixpeaks
from heart_rate.heart_rate_fragmentation import compute_HRF

**RMSSD** is calculated on 5-minute windows as extensively done in the literature (e.g., https://pmc.ncbi.nlm.nih.gov/articles/PMC10566244/). Usually, consecutive windows are used, but since my computation is quite fast I am now opting for an overlap of 4 minutes (step of 1 min) between windows to increase the granularity. Moreover, in this way, I do not lose the last part of my quite portion (I loose at max 1 min). This choice can be discussed (PS I tried with 1s step and results seem interesing....)

**SDNN** I don't remember what Silvani said it's better to do (I think 5 min windows also for this)

For **heart rate fragmentation**, the longer the window the better (https://physionet.org/content/heart-rate-fragmentation-code/1.0.0/)



In [4]:
threshold_bursts = 35/1000 # threshold for detecting bursts in mg (validated)
window_length = pd.Timedelta("5 min")  # window length
window_step = pd.Timedelta("1 min")  # window step

HRV = []  # Reset HRV storage

ibi_quiet_all = []

for i, (start_sleep, end_sleep) in enumerate(nights):  # for each night

    acc_night = compute_acc_SMV(acc_df.loc[start_sleep:end_sleep])
    ppg_night = ppg_df.loc[start_sleep:end_sleep]

    # Detect wrist accelerometer bursts
    bursts = detect_bursts(acc_night, sampling_rate=64, alfa=threshold_bursts)

    # Extract quiet periods (no movement of the wrist)
    quiet_periods = pd.DataFrame()
    quiet_periods["start"] = bursts["end"].iloc[:-1].reset_index(drop=True)
    quiet_periods["end"] = bursts["start"].iloc[1:].reset_index(drop=True)

    for _, quiet_period in quiet_periods.iterrows():  # for each quiet period

        duration_quiet_period = quiet_period["end"] - quiet_period["start"]

        if duration_quiet_period < window_length:  # If the whole period is shorter than 5 min, skip it
            continue
            
        acc_quiet = acc_night.loc[quiet_period["start"]:quiet_period["end"]]
        ppg_quiet = ppg_night.loc[quiet_period["start"]:quiet_period["end"]]

         # Extract systolic peaks from the quiet PPG signal
        _, peaks = MSPTDfast(ppg_quiet["ppg"].values, sampling_rate=64)
        t_peaks = ppg_quiet.index.to_series().values[peaks]
        ibi = np.diff(t_peaks).astype('timedelta64[ns]').astype('float64') / 1e9  # seconds
        ibi = np.insert(ibi, 0, np.mean(ibi[1:10]), axis=0)  # Set first value as mean of next 10
        ibi = pd.Series(ibi, index=t_peaks)

        # Kubios artifact correction
        artifacts, env_diff_corrected = signal_fixpeaks(ibi.values, 64, iterative=False)
        artifacts_all = np.concatenate((artifacts["ectopic"], artifacts["missed"], artifacts["extra"], artifacts["longshort"]))
        ibi[ibi.index[artifacts_all.astype(int)]] = np.nan
        ibi_clean = ibi.interpolate(method="linear")

        # Generate overlapping windows of 5 minutes with 30-second overlap
        current_start = quiet_period["start"]
        
        # For each window
        while current_start + window_length <= quiet_period["end"]:

            current_end = current_start + window_length

            ibi_window = ibi_clean.loc[current_start:current_end]

            # HRV Features
            ppi = ibi_window.values * 1000  # Convert to ms
            diff_ppi = np.diff(ppi)

            rmssd = np.sqrt(np.mean(diff_ppi**2))  # RMSSD
            sdnn = np.std(ppi, ddof=1)  # SDNN
            PIP = compute_HRF(ppi)  # Custom HRF computation

            HRV.append({
                "day": i+1,
                "time": current_start + window_length / 2,
                "rmssd": rmssd, 
                "sdnn": sdnn, 
                "PIP": PIP
            })

            current_start += window_step  # Move to next overlapping window

        ibi_quiet_all.append(ibi_clean)

HRV_df = pd.DataFrame(HRV)
ibi_quiet_df = pd.concat(ibi_quiet_all)

In [ ]:
from visualization.plot_HRV import plot_HRV

# Decide whether to plot in notebook or in a separate window
from bokeh.plotting import output_notebook
output_notebook()

plot_HRV(ibi_quiet_df, HRV_df)